In [2]:
import tensorflow as tf

In [3]:
from __future__ import division, print_function, absolute_import

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
# Parametry trenujące 

learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step =  10

# Parametry sieci

num_input = 784 # Wymiary danych wejsciowych( poniewaz wielkosc obrazu to 28*28px)
num_classes = 10 # tyle klas do rozpoznania (digits 0-9)
dropout = .75# Dropout, prawdopodobieństwo do zatrzymania jednostek

X =  tf.placeholder(tf.float32, [None, num_input])
Y =  tf.placeholder(tf.float32, [None, num_classes])
keep_prob =  tf.placeholder(tf.float32)

In [14]:
# Create some wrappers for simplicity

def conv2d(x, W, b, strides = 1):
        x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding ='SAME')
        x = tf.nn.bias_add(x, b)
        return tf.nn.relu(x)
def maxpool2d(x, k = 2):
    
    return tf.nn.max_pool(x, ksize = [1, k, k, 1], 
                         strides = [1, k, k, 1], 
                         padding = 'SAME')


# Tworzenie modelu

def conv_net(x, weight, biases, dropout):
    
    x = tf.reshape(x, shape = [-1, 28,28, 1])
    
    conv1 = conv2d(x, weight['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Convolutional layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    
    # Fully connected layer
    # Reshape conv2 output to fit fully conected layer input
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 =  tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output class prediction
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return(out)

In [15]:
# Przetrzymywanie wag oraz obciazen 

weights = {
    
    # 5x5 conv, 1input, 32 outputs
    'wc1':tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    # 5x5 conv, 32, inputs, 64 outputs 
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    # fully conected , 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), 
    'out' : tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    
    'bc1': tf.Variable(tf.random_normal([32])), 
    'bc2': tf.Variable(tf.random_normal([64])), 
    'bd1': tf.Variable(tf.random_normal([1024])), 
    'out': tf.Variable(tf.random_normal([num_classes]))}


# Tworzenie modelu


logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)


# Zdefiniowanie funkcji straty

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, 
                                                                labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)


# Przetworzenie modelu

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize
init = tf.global_variables_initializer()

In [ ]:
# Rozpoczecie trenowania

with tf.Session() as sess:
    sess.run(init)
    
    
    
    for step in range(1, num_steps+1):
        
        batch_x, batch_y =  mnist.train.next_batch(batch_size)
        
        sess.run(train_op, feed_dict = {X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step --1:
            
            loss, acc= sess.run([loss_op, accuracy], feed_dict={X: batch_x, 
                                                               Y: batch_y, 
                                                               keep_prob: 1.0})
            
            print("Step " + str(step)+ " Minibatch Loss =" +                   "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        print("Optimization Finisherd")
        
        
        print("test acc", 
             sess.run(accuracy, feed_dict={X: mnist.test.images[:256], 
                                          Y: mnist.test.labels[:256], 
                                          keep_prob: 1.0}))

Step 1 Minibatch Loss =609116.3750, Training Accuracy= 0.242
Optimization Finisherd
test acc 0.148438
Step 2 Minibatch Loss =401980.6562, Training Accuracy= 0.195
Optimization Finisherd
test acc 0.171875
Step 3 Minibatch Loss =88258.6406, Training Accuracy= 0.102
Optimization Finisherd
test acc 0.0976563
Step 4 Minibatch Loss =4791.4014, Training Accuracy= 0.094
Optimization Finisherd
test acc 0.0820313
Step 5 Minibatch Loss =557.0072, Training Accuracy= 0.109
Optimization Finisherd
test acc 0.0976563
Step 6 Minibatch Loss =85.3465, Training Accuracy= 0.109
Optimization Finisherd
test acc 0.0585938
Step 7 Minibatch Loss =63.1277, Training Accuracy= 0.055
Optimization Finisherd
test acc 0.125
Step 8 Minibatch Loss =112.3827, Training Accuracy= 0.109
Optimization Finisherd
test acc 0.136719
Step 9 Minibatch Loss =58.9983, Training Accuracy= 0.109
Optimization Finisherd
test acc 0.140625
Step 10 Minibatch Loss =77.7111, Training Accuracy= 0.133
Optimization Finisherd
test acc 0.0976563
St

test acc 0.121094
Step 85 Minibatch Loss =2.4900, Training Accuracy= 0.148
Optimization Finisherd
test acc 0.109375
Step 86 Minibatch Loss =2.4506, Training Accuracy= 0.180
Optimization Finisherd
test acc 0.0859375
Step 87 Minibatch Loss =2.6059, Training Accuracy= 0.086
Optimization Finisherd
test acc 0.0859375
Step 88 Minibatch Loss =2.3710, Training Accuracy= 0.094
Optimization Finisherd
test acc 0.101563
Step 89 Minibatch Loss =2.4017, Training Accuracy= 0.117
Optimization Finisherd
test acc 0.101563
Step 90 Minibatch Loss =2.4070, Training Accuracy= 0.117
Optimization Finisherd
test acc 0.0898438
Step 91 Minibatch Loss =2.4212, Training Accuracy= 0.141
Optimization Finisherd
test acc 0.0742188
Step 92 Minibatch Loss =2.3411, Training Accuracy= 0.125
Optimization Finisherd
test acc 0.0898438
Step 93 Minibatch Loss =2.4060, Training Accuracy= 0.164
Optimization Finisherd
test acc 0.136719
Step 94 Minibatch Loss =2.4212, Training Accuracy= 0.141
Optimization Finisherd
test acc 0.1367